# time데이터 분석

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
base_path = './preprocessed data/'

train_err = pd.read_csv(base_path + 'new_train_err.csv', parse_dates=['time'])
train_quality = pd.read_csv(base_path + 'new_train_quality.csv', parse_dates=['time'])
train_problem = pd.read_csv(base_path + 'new_train_problem.csv', parse_dates=['time'])
test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv', parse_dates=['time'])
# submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

In [4]:
train_err['day'] = train_err.time.dt.day
train_err['hour'] = train_err.time.dt.hour
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode,day,hour
0,10000,2020-11-01 02:56:16,model_3,05.15.2138,15,1,1,2
1,10000,2020-11-01 03:03:09,model_3,05.15.2138,12,1,1,3
2,10000,2020-11-01 03:03:09,model_3,05.15.2138,11,1,1,3
3,10000,2020-11-01 05:05:14,model_3,05.15.2138,16,1,1,5
4,10000,2020-11-01 05:05:15,model_3,05.15.2138,4,0,1,5


In [8]:
train_problem['day'] = train_problem.time.dt.day
train_problem['hour'] = train_problem.time.dt.hour
train_problem = train_problem.sort_values('user_id')
train_problem.head()

,user_id,time,day,hour
1116,10001,2020-11-13 19:00:00,13,19
492,10004,2020-11-04 12:00:00,4,12
3857,10005,2020-11-13 12:00:00,13,12
3826,10006,2020-11-10 18:00:00,10,18
414,10006,2020-11-10 17:00:00,10,17


## 에러가 발생한 기간에 따른 불만 제기율

In [60]:
user_day_df = train_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df

,user_id,day
0,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,10001,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,10002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,10003,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,10004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]"
14996,24996,[30]
14997,24997,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
14998,24998,"[1, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19..."


In [61]:
user_day_df['period'] = 0 
user_day_df

,user_id,day,period
0,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
1,10001,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
2,10002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
3,10003,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
4,10004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
...,...,...,...
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]",0
14996,24996,[30],0
14997,24997,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0
14998,24998,"[1, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19...",0


In [69]:
days = 0
for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

user_day_df

,user_id,day,period
0,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
1,10001,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
2,10002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
3,10003,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
4,10004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]",10
14996,24996,[30],1
14997,24997,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14998,24998,"[1, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 18, 19...",21


In [85]:
# 에러 발생 기간이 20일 미만
df1 = user_day_df[user_day_df['period'] < 20]
df1

,user_id,day,period
8,10008,"[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 2...",18
16,10016,[6],1
17,10017,"[26, 27, 28, 29, 30]",5
18,10018,"[2, 5, 7, 9, 11, 12, 14, 16, 18]",9
28,10028,"[5, 6, 13, 14, 15, 16, 18, 20, 22, 24, 30]",11
...,...,...,...
14982,24982,[9],1
14989,24989,[13],1
14991,24991,"[1, 3, 4, 5, 7, 9, 11, 13, 15, 17, 18, 19, 21,...",17
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]",10


In [86]:
# 14%
print(len(train_problem[train_problem.user_id.isin(df1.user_id)].user_id.unique()))
print(round(187/1315*100, 2))

187
14.22


In [87]:
# 평균 기간보다 짧게 오류가 나온 사람
df2 = user_day_df[user_day_df['period'] < 28]
df2

,user_id,day,period
6,10006,"[1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, ...",27
7,10007,"[1, 2, 3, 4, 5, 7, 9, 11, 13, 14, 15, 17, 19, ...",22
8,10008,"[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 2...",18
16,10016,[6],1
17,10017,"[26, 27, 28, 29, 30]",5
...,...,...,...
14991,24991,"[1, 3, 4, 5, 7, 9, 11, 13, 15, 17, 18, 19, 21,...",17
14992,24992,"[2, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 17, ...",22
14995,24995,"[20, 21, 22, 23, 25, 26, 27, 28, 29, 30]",10
14996,24996,[30],1


In [89]:
# 22%

print(len(train_problem[train_problem.user_id.isin(df2.user_id)].user_id.unique()))
print(round(712/3217*100, 2))

712
22.13


In [91]:
# 평균 기간보다 더 길게 오류가 나온 사람
df3 = user_day_df[user_day_df['period'] > 27]
df3

,user_id,day,period
0,10000,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
1,10001,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
2,10002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
3,10003,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
4,10004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14990,24990,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14993,24993,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14994,24994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",29
14997,24997,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30


In [92]:
# 36%
print(len(train_problem[train_problem.user_id.isin(df3.user_id)].user_id.unique()))
print(round(4288/11783*100, 2))

4288
36.39
